Importing necessary libraries and modules

In [ ]:
from functools import partial
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

In [ ]:
!pip install torchtext==0.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1

Installing required packages

In [ ]:
!pip install 'portalocker>=2.0.0'
import portalocker

Importing torchtext modules

In [ ]:
from torchtext import datasets
#from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


Setting up the device (GPU or CPU)

In [ ]:
if torch.cuda.is_available():
    device=torch.device(type='cuda',index=0)
else:
    device=torch.device(type='cpu',index=0)

In [ ]:
train_data = datasets.IMDB(split='train')  # data is ShardingFilterIterDataPipe
# DataPipe that yields tuple of label 1 or 2 and text containing the movie review

eval_data = datasets.IMDB(split='test')

mapped_train_data = []
for label, review in train_data:
    mapped_train_data.append(review)  # we just need reviews for word2vec

eval_data_list = list(eval_data)  # convert eval_data to a list
mapped_eval_data = []
for label, review in eval_data_list[:5000]:
    mapped_eval_data.append(review)

mapped_train_data[0]  # checking

print(type(mapped_train_data[0]))  # string

mapped_train_data[0:2]  # list of string

tokenizer = get_tokenizer("basic_english", language="en")
print(mapped_train_data[0:2])

<class 'str'>
['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and 

Building the vocabulary

In [ ]:
#build the vocab now
min_word_freq=20
def build_vocab(mapped_train_data, tokenizer):
    vocab = build_vocab_from_iterator(
        map(tokenizer, mapped_train_data),
        specials=["<unk>"],
        min_freq=min_word_freq
    )
    vocab.set_default_index(vocab["<unk>"])
    return vocab

In [ ]:
vocab=build_vocab(mapped_train_data,tokenizer)  #Creating the vocabulary
vocab_size=len(vocab)
print(vocab_size)

#Defining hyperparameters
window_size=4 #leads to context window size of 9
max_seq_len=256
max_norm=1
embed_dim=300
batch_size=64
text_pipeline = lambda x: vocab(tokenizer(x))
sample=text_pipeline("Hello World")
print(sample)
print(type(sample))

13351
[4644, 185]
<class 'list'>


Defining the collate_cbow function

In [ ]:
def collate_cbow(batch, text_pipeline):

     batch_input_words, batch_target_word = [], []

     for review in batch:

         review_tokens_ids = text_pipeline(review)

         if len(review_tokens_ids) < window_size * 2 + 1:
             continue

         if max_seq_len:
             review_tokens_ids = review_tokens_ids[:max_seq_len]

         for idx in range(len(review_tokens_ids) - window_size * 2):
             current_ids_sequence = review_tokens_ids[idx : (idx + window_size * 2 + 1)]
             target_word = current_ids_sequence.pop(window_size)
             input_words = current_ids_sequence
             batch_input_words.append(input_words)
             batch_target_word.append(target_word)

     batch_input_words = torch.tensor(batch_input_words, dtype=torch.long)
     batch_target_word = torch.tensor(batch_target_word, dtype=torch.long)

     return batch_input_words, batch_target_word

Defining the collate_skipgram function

In [ ]:
def collate_skipgram(batch, text_pipeline):

    batch_input_word, batch_target_words = [], []

    for review in batch:
        review_tokens_ids = text_pipeline(review)
        if len(review_tokens_ids) < window_size * 2 + 1:
            continue
        if max_seq_len:
            review_tokens_ids = review_tokens_ids[:max_seq_len]
        for idx in range(len(review_tokens_ids) - window_size * 2):
            current_ids_sequence = review_tokens_ids[idx : (idx + window_size * 2 + 1)]
            input_word = current_ids_sequence.pop(window_size)
            target_words = current_ids_sequence

            for target_word in target_words:
                batch_input_word.append(input_word)
                batch_target_words.append(target_word)

    batch_input_word = torch.tensor(batch_input_word, dtype=torch.long)
    batch_target_words = torch.tensor(batch_target_words, dtype=torch.long)
    return batch_input_word, batch_target_words

Creating DataLoaders for CBOW and Skip-Gram models

In [ ]:
#Training DataLoaders
traindl_cbow = DataLoader(
        mapped_train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=partial(collate_cbow,text_pipeline=text_pipeline)
    )


traindl_skipgram = DataLoader(
        mapped_train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=partial(collate_skipgram,text_pipeline=text_pipeline)
    )

Evaluation DataLoaders

In [ ]:
evaldl_cbow = DataLoader(
        mapped_eval_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=partial(collate_cbow,text_pipeline=text_pipeline)
    )

evaldl_skipgram = DataLoader(
        mapped_eval_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=partial(collate_skipgram,text_pipeline=text_pipeline)
    )

Defining the CBOW class

In [ ]:
class CBOW(nn.Module):
    def __init__(self, vocab_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim)
        # Other layers or initialization code
        self.linear = nn.Linear(
            in_features=embed_dim,
            out_features=vocab_size,
        )

#Forward pass
    def forward(self, x):
        #print("Shape of x before embedding:",x.shape)
        x = self.embeddings(x)
        #print("Shape of x after embedding:",x.shape)
        x = x.mean(axis=1)
        #print("Shape of x after mean:",x.shape)
        x = self.linear(x)
        #print("Shape of x at the end of forward:",x.shape)
        return x


Defining the SkipGram class

In [ ]:
class SkipGram(nn.Module):

    def _init_(self, vocab_size):
        super()._init_()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_dim,
            max_norm=max_norm
        )
        self.linear = nn.Linear(
            in_features=embed_dim,
            out_features=vocab_size,
        )

#Forward pass
    def forward(self, x):
        #print("Shape of x before embedding:",x.shape)
        x = self.embeddings(x)
        #print("Shape of x after embedding:",x.shape)
        x = self.linear(x)
        #print("Shape of x at the end of forward:",x.shape)
        return x

Defining the train_one_epoch function

In [ ]:
def train_one_epoch(model,dataloader):
    model.train()
    running_loss = []

    for i, batch_data in enumerate(dataloader):
        inputs = batch_data[0].to(device)
        targets = batch_data[1].to(device)
        #print("Input Shape:",inputs.shape, "Target Shape:",targets.shape)
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        opt.step()

        running_loss.append(loss.item())

    epoch_loss = np.mean(running_loss)
    print("Train Epoch Loss:",round(epoch_loss,3))
    loss_dict["train"].append(epoch_loss)

Defining the validate_one_epoch function

In [ ]:
def validate_one_epoch(model,dataloader):
    model.eval()
    running_loss = []

    with torch.no_grad():
        for i, batch_data in enumerate(dataloader, 1):
            inputs = batch_data[0].to(device)
            targets = batch_data[1].to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            running_loss.append(loss.item())


    epoch_loss = np.mean(running_loss)
    #print("Validation Epoch Loss:",round(epoch_loss,3))
    loss_dict["val"].append(epoch_loss)

loss_fn=nn.CrossEntropyLoss()
n_epochs=1
loss_dict={}
loss_dict["train"]=[]
loss_dict["val"]=[]

choice=input("Enter cbow/skipgram:")
if choice=="cbow":
    model=CBOW(vocab_size).to(device)
    dataloader_train=traindl_cbow
    dataloader_val=evaldl_cbow
elif choice=="skipgram":
    model=SkipGram(vocab_size).to(device)
    dataloader_train=traindl_skipgram
    dataloader_val=evaldl_skipgram

opt=optim.Adam(params=model.parameters(),lr=0.001)

for e in range(n_epochs):
    print("Epoch=",e+1)
    train_one_epoch(model,dataloader_train)
    validate_one_epoch(model,dataloader_val)

for name,child in model.named_children():
    print(name,child)

trimmed_model=model.embeddings
print(trimmed_model)

print(vocab.get_itos())
print(vocab.lookup_indices(["film","movie"]))

emb1=trimmed_model(torch.tensor([23]))
emb2=trimmed_model(torch.tensor([17]))
cos=torch.nn.CosineSimilarity()
print(cos(emb1,emb2))


Enter cbow/skipgram:cbow
Epoch= 1


KeyboardInterrupt: 